In [ ]:

import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import joblib


In [ ]:

# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Función para preprocesar texto
def preprocesar(texto):
    doc = nlp(texto.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])


In [ ]:

# Leer archivo CSV y aplicar preprocesamiento
df = pd.read_csv('./activos_pi_2.csv', encoding='latin1')
df["descripcion"] = df["descripcion"].apply(preprocesar)

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(df["descripcion"], df["categoria"], test_size=0.2, random_state=42)

# Vectorización TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:

# Diccionario de modelos
modelos = {
    "SVM": SVC(kernel='linear', C=1.0, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42)
}

# Entrenamiento y evaluación
for nombre, modelo in modelos.items():
    modelo.fit(X_train_tfidf, y_train)
    y_pred = modelo.predict(X_test_tfidf)

    print(f"=== Reporte de Clasificación - {nombre} ===")
    print(classification_report(y_test, y_pred))

    cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=modelo.classes_)
    disp.plot(xticks_rotation=45)
    plt.title(f"Matriz de Confusión - {nombre}")
    plt.tight_layout()
    plt.show()

    # Guardar modelo
    joblib.dump(modelo, f"modelo_{nombre.lower().replace(' ', '_')}.pkl")

# Guardar vectorizador
joblib.dump(vectorizer, "vectorizer_tfidf.pkl")


In [ ]:

# Prueba con nueva descripción
nueva_descripcion = ["un dispositivo mejorado para la comunicación"]
descripcion_proc = [preprocesar(nueva_descripcion[0])]
nueva_tfidf = vectorizer.transform(descripcion_proc)

# Cargar y probar cada modelo
for nombre in modelos:
    modelo_cargado = joblib.load(f"modelo_{nombre.lower().replace(' ', '_')}.pkl")
    prediccion = modelo_cargado.predict(nueva_tfidf)[0]
    print(f"Predicción con {nombre}: {prediccion}")
